In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]


In [9]:

for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [10]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [11]:
tasks = 0

In [119]:
pd.read_sql(
    '''
    SELECT * FROM Orders
    ''',
    con,
)

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,04.07.1996,3
1,10249,81,6,05.07.1996,1
2,10250,34,4,08.07.1996,2
3,10251,84,3,08.07.1996,1
4,10252,76,4,09.07.1996,2
...,...,...,...,...,...
120,10368,20,2,29.11.1996,2
121,10369,75,8,02.12.1996,2
122,10370,14,6,03.12.1996,2
123,10371,41,1,03.12.1996,1


1.Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [203]:
tasks=0
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM Customers 
    where Country = 'Germany' or Country ='France' or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2.Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [204]:
tasks+=1
pd.read_sql(
    '''
    SELECT count(Country), Country FROM Customers
    group by Country
    order by count(Country) desc
    LIMIT 3
    
    ''',
    con,
)


,count(Country),Country
0,13,USA
1,11,Germany
2,11,France


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [205]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName,OrderDate FROM Shippers
    join Orders on Shippers.ShipperID = Orders.ShipperID
    order by OrderDate
    limit 1 offset 9
    
    ''',
    con,
)

,ShipperName,OrderDate
0,United Package,03.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [206]:
tasks+=1
pd.read_sql(
    '''
    WITH temporarytable AS (SELECT OrderID, sum(Price) as TotalPrice FROM Products
    join OrderDetails on Products.ProductID = OrderDetails.ProductID
    group by OrderID
    order by TotalPrice desc
    limit 1)
    SELECT t.OrderID, Products.ProductName, Products.Price FROM temporarytable t
    join OrderDetails on  OrderDetails.OrderID = t.OrderID
    join Products on Products.ProductID = OrderDetails.ProductID
    ''',
    con,
)

,OrderID,ProductName,Price
0,10360,Rössle Sauerkraut,45.60
1,10360,Thüringer Rostbratwurst,123.79
2,10360,Côte de Blaye,263.50
3,10360,Maxilaku,20.00
4,10360,Tourtière,7.45


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [207]:
tasks+=1
pd.read_sql(
    '''
    WITH temporarytable AS (SELECT OrderDetails.ProductID, sum(Quantity) as TotalQuantity, ProductName FROM OrderDetails
    join Products on OrderDetails.ProductID = Products.ProductID
    group by OrderDetails.ProductID
    order by sum(OrderDetails.Quantity) desc
    limit 1
    
    )
    SELECT ProductName, OrderID, Quantity, TotalQuantity  FROM temporarytable
    join OrderDetails on temporarytable.ProductID = OrderDetails.ProductID

    
    ''',
    con,
)

,ProductName,OrderID,Quantity,TotalQuantity
0,Gorgonzola Telino,10253,20,458
1,Gorgonzola Telino,10272,40,458
2,Gorgonzola Telino,10273,15,458
3,Gorgonzola Telino,10325,4,458
4,Gorgonzola Telino,10335,25,458
5,Gorgonzola Telino,10342,56,458
6,Gorgonzola Telino,10356,30,458
7,Gorgonzola Telino,10359,70,458
8,Gorgonzola Telino,10363,20,458
9,Gorgonzola Telino,10374,30,458


6.Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [208]:
tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName	, Country, ContactName, Phone FROM Products
    join OrderDetails on OrderDetails.ProductID = Products.ProductID
    
    join Suppliers on Suppliers.SupplierID = Products.SupplierID
    group by Products.SupplierID
    order by count(OrderID) desc
    limit 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7.Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [209]:
tasks+=1
pd.read_sql(
    '''
    SELECT Customers.Country, CategoryName, sum(Price) as TotalSum FROM Customers 
    join Orders on Orders.CustomerID = Customers.CustomerID
    join OrderDetails on OrderDetails.OrderID = Orders.OrderID
    join Products on OrderDetails.ProductID = Products.ProductID
    join Categories on Categories.CategoryID = Products.CategoryID
    where Customers.Country = 'Brazil'
    group by CategoryName
    order by TotalSum desc
    limit 1
    ''',
    con,
)

,Country,CategoryName,TotalSum
0,Brazil,Beverages,354.25


8.Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [210]:
tasks+=1
pd.read_sql(
    '''
     WITH temporarytable AS (SELECT OrderDetails.OrderID, Customers.Country, sum(Price) as OrderCost FROM Customers 
    join Orders on Orders.CustomerID = Customers.CustomerID
    join OrderDetails on OrderDetails.OrderID = Orders.OrderID
    join Products on OrderDetails.ProductID = Products.ProductID

    where Customers.Country = 'USA'
    group by OrderDetails.OrderID
    )
    SELECT max(OrderCost)-min(OrderCost)  as Difference FROM temporarytable 
   
    ''',
    con,
)

,Difference
0,334.09


9.Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [211]:
tasks+=1
pd.read_sql(
    '''
    
    SELECT  count(OrderID) as OrderCount, FirstName || ' ' || LastName as Name  FROM Employees 
    left join Orders on Orders.EmployeeID = Employees.EmployeeID
    
    group by Orders.EmployeeID
    order by BirthDate
    limit 3    
    
        ''',
    con,
)

,OrderCount,Name
0,13,Michael Suyama
1,4,Anne Dodsworth
2,9,Steven Buchanan


10.Сколько банок крабового мяса всего было заказано.

In [212]:
tasks+=1
pd.read_sql(
    '''
    
    SELECT ProductName, sum(Quantity) as TotalQuantity FROM Orders    
    join OrderDetails on OrderDetails.OrderID = Orders.OrderID
    join Products on OrderDetails.ProductID = Products.ProductID

    where Products.ProductName like '%crab%'
    group by Products.ProductID
    
   
    ''',
    con,
)

,ProductName,TotalQuantity
0,Boston Crab Meat,204


In [213]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
